In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math as math
import sklearn as sklearn
import random as random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import LeaveOneOut
%matplotlib notebook
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import rand_score
from IPython.display import clear_output
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow import feature_column

In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [277]:
data_test = pd.read_csv('unlabeled.csv') 
data_test.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,2,127112,7,NaN,56.0,21.102439,0,Caucasian,M,180.3,Emergency Department,Accident & Emergency,1105,admit,Neuro ICU,0.104167,0,68.6,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,NaN,NaN,74.0,51.0,74.0,51.0,100.0,78.0,NaN,NaN,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,NaN,NaN,121.0,79.0,121.0,79.0,37.1,36.6,NaN,NaN,55.0,51.0,55.0,51.0,88.0,84.0,NaN,NaN,68.0,60.0,68.0,60.0,18.0,8.0,100.0,98.0,NaN,NaN,91.0,79.0,91.0,79.0,36.6,36.6,3.7,3.7,NaN,NaN,19.0,19.0,9.3,9.3,0.67,0.67,276.0,167.0,24.0,24.0,13.8,13.8,42.0,42.0,NaN,NaN,NaN,NaN,223.0,223.0,4.8,4.8,135.0,134.0,4.7,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
1,5,130737,178,NaN,NaN,19.500595,1,Caucasian,F,145.0,Operating Room,Operating 

In [278]:
data_test_clean=data_test.drop("readmission_status", axis='columns') #all of them are nan values so we drop

In [279]:
data_test_clean=data_test_clean.drop("gender", axis='columns') #there is no real difference between male and female
#death correlation so it doesn't influence =» we drop

In [280]:
data_test_clean=data_test_clean.drop("ethnicity", axis='columns')#there is no real difference between ethnicities
#death correlation so it doesn't influence =» we drop

In [281]:
data_test_clean=data_test_clean.drop("patient_id", axis='columns')

In [282]:
data_test_clean=data_test_clean.drop("encounter_id", axis='columns')

In [283]:
data_test_clean=data_test_clean.drop("bmi", axis='columns') #even if the first kde seems to give us a lot of difference,
#in a normalised form we can see that no matter the bmi, dead vs. alive is kind of the same. =»

In [284]:
data_test_clean=data_test_clean.drop("weight", axis='columns')

In [285]:
data_test_clean=data_test_clean.drop("height", axis='columns')

In [286]:
data_test_clean=data_test_clean.drop("icu_stay_type", axis='columns') #the ICU type is more relevant than the stay type bc it
#is more varied

In [287]:
data_test_clean=data_test_clean.drop("pre_icu_los_days", axis='columns') #not really enough difference

In [288]:
data_test_clean=data_test_clean.drop("apache_2_bodysystem", axis='columns') #they are kind of the same as apache 3

In [289]:
data_test_clean=data_test_clean.drop("apache_4a_hospital_death_prob", axis='columns')

In [290]:
mask = data_test_clean[['aids', 'leukemia', 'cirrhosis','diabetes_mellitus',
                     'hepatic_failure','immunosuppression','lymphoma',
                                'solid_tumor_with_metastasis']].notna().all(axis=1)
data_test_clean = data_test_clean[mask]
#bc .dropna makes it so that I can't use .head() or .shape

In [291]:
data_test_clean=data_test_clean.drop(["d1_arterial_pco2_max","d1_arterial_pco2_min","d1_arterial_ph_max",
                                       "d1_arterial_ph_min","d1_arterial_po2_max","d1_arterial_po2_min",
                                       "d1_pao2fio2ratio_max","d1_pao2fio2ratio_min"],axis=1)

In [292]:
data_test_clean=data_test_clean.drop(["apache_3j_diagnosis","gcs_eyes_apache","gcs_motor_apache",
                                       "gcs_verbal_apache","paco2_apache"],axis=1)

In [293]:
data_test_clean['d1_mbp_invasive_max']=data_test_clean['d1_mbp_invasive_max'].fillna(data_test_clean['d1_mbp_noninvasive_max'])

In [294]:
data_test_clean=data_test_clean.drop("d1_mbp_noninvasive_max", axis='columns')

In [295]:
data_test_clean['d1_diasbp_invasive_max']=data_test_clean['d1_diasbp_invasive_max'].fillna(data_test_clean['d1_diasbp_noninvasive_max'])

In [296]:
data_test_clean=data_test_clean.drop("d1_diasbp_noninvasive_max", axis='columns')

In [297]:
data_test_clean['d1_diasbp_invasive_min']=data_test_clean['d1_diasbp_invasive_min'].fillna(data_test_clean['d1_diasbp_noninvasive_min'])

In [298]:
data_test_clean=data_test_clean.drop("d1_diasbp_noninvasive_min", axis='columns')

In [299]:
data_test_clean['d1_mbp_invasive_min']=data_test_clean['d1_mbp_invasive_min'].fillna(data_test_clean['d1_mbp_noninvasive_min'])

In [300]:
data_test_clean=data_test_clean.drop("d1_mbp_noninvasive_min", axis='columns')

In [301]:
data_test_clean['d1_sysbp_invasive_max']=data_test_clean['d1_sysbp_invasive_max'].fillna(data_test_clean['d1_sysbp_noninvasive_max'])

In [302]:
data_test_clean=data_test_clean.drop("d1_sysbp_noninvasive_max", axis='columns')

In [303]:
data_test_clean['d1_sysbp_invasive_min']=data_test_clean['d1_sysbp_invasive_min'].fillna(data_test_clean['d1_sysbp_noninvasive_min'])

In [304]:
data_test_clean=data_test_clean.drop("d1_sysbp_noninvasive_min", axis='columns')

In [305]:
data_test_clean['h1_diasbp_invasive_max']=data_test_clean['h1_diasbp_invasive_max'].fillna(data_test_clean['h1_diasbp_noninvasive_max'])

In [306]:
data_test_clean=data_test_clean.drop("h1_diasbp_noninvasive_max", axis='columns')

In [307]:
data_test_clean['h1_diasbp_invasive_min']=data_test_clean['h1_diasbp_invasive_min'].fillna(data_test_clean['h1_diasbp_noninvasive_min'])

In [308]:
data_test_clean=data_test_clean.drop("h1_diasbp_noninvasive_min", axis='columns')

In [309]:
data_test_clean['h1_mbp_invasive_max']=data_test_clean['h1_mbp_invasive_max'].fillna(data_test_clean['h1_mbp_noninvasive_max'])

In [310]:
data_test_clean=data_test_clean.drop("h1_mbp_noninvasive_max", axis='columns')

In [311]:
data_test_clean['h1_sysbp_invasive_max']=data_test_clean['h1_sysbp_invasive_max'].fillna(data_test_clean['h1_sysbp_noninvasive_max'])

In [312]:
data_test_clean=data_test_clean.drop("h1_sysbp_noninvasive_max", axis='columns')

In [313]:
data_test_clean['h1_sysbp_invasive_min']=data_test_clean['h1_sysbp_invasive_min'].fillna(data_test_clean['h1_sysbp_noninvasive_min'])

In [314]:
data_test_clean=data_test_clean.drop("h1_sysbp_noninvasive_min", axis='columns')

In [315]:
data_test_clean=data_test_clean.drop(['d1_albumin_max','d1_albumin_min','d1_bilirubin_max','d1_bilirubin_min',
'd1_bun_max','d1_bun_min','d1_calcium_max','d1_calcium_min','d1_creatinine_max','d1_creatinine_min',
'd1_glucose_max','d1_glucose_min','d1_hco3_max','d1_hco3_min','d1_hemaglobin_max','d1_hemaglobin_min',
'd1_hematocrit_max','d1_hematocrit_min','d1_inr_max','d1_inr_min','d1_lactate_max','d1_lactate_min',
'd1_platelets_max','d1_platelets_min','d1_potassium_max','d1_potassium_min','d1_sodium_max',
'd1_sodium_min','d1_wbc_max','d1_wbc_min'], axis=1)

In [316]:
data_test_clean.shape

(39129, 120)

In [317]:
data_test_clean.head()

,hospital_id,hospital_death,age,elective_surgery,hospital_admit_source,icu_admit_source,icu_id,icu_type,albumin_apache,apache_2_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_unable_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_temp_max,h1_temp_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem
0,7,NaN,56.0,0,Emergency Department,Accident & Emergency,1105,Neuro ICU,3.7,301.0,0,0.0,NaN,19.0,0.67,NaN,0.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,74.0,51.0,74.0,51.0,100.0,78.0,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,121.0,79.0,121.0,79.0,37.1,36.6,55.0,51.0,55.0,51.0,88.0,84.0,68.0,NaN,68.0,60.0,60.0,18.0,8.0,100.0,98.0,91.0,79.0,91.0,79.0,36.6,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological
1,178,NaN,NaN,1,Operating Room,Operating Room / Recovery,1043,Med-Surg ICU,NaN,217.0,1,0.0,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,0.0,53.0,NaN,NaN,NaN,37.0,NaN,37.0,843.4368,0.0,NaN,65.0,31.0,65.0,31.0,73.0,60.0,108.0,53.0,108.0,53.0,33.0,16.0,99.0,90.0,148.0,83.0,148.0,83.0,37.7,36.6,50.0,44.0,50.0,44.0,60.0,60.0,80.0,NaN,80.0,80.0,80.0,16.0,16.0,94.0,92.0,121.0,96.0,121.0,96.0,36.6,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological
2,86,NaN,66.0,1,Operating Room,Operating Room / Recovery,1035,CCU-CTICU,NaN,302.0,1,0.0,NaN,22.0,1.84,NaN,0.0,81.0,124.0,39.2,0.0,115.0,NaN,NaN,NaN,31.0,130.0,35.5,1728.0000,0.0,14.8,85.0,57.0,85.0,57.0,117.0,86.0,110.0,71.0,110.0,71.0,28.0,10.0,100.0,95.0,172.0,102.0,172.0,102.0,37.5,35.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular
3,163,NaN,64.0,0,Operating Room,Operating Room / Recovery,1078,Med-Surg ICU,1.9,303.0,0,0.0,0.8,24.0,0.75,0.7,0.0,122.0,111.0,32.0,1.0,48.0,36.0,142.0,7.35,30.0,133.0,37.7,1434.4128,1.0,9.7,63.0,42.0,63.0,42.0,111.0,60.0,76.0,48.0,76.0,48.0,35.0,15.0,100.0,92.0,115.0,69.0,115.0,69.0,38.4,37.5,62.0,57.0,62.0,57.0,111.0,102.0,76.0,NaN,76.0,65.0,65.0,26.

In [318]:
categorical_features = ["hospital_id", "hospital_admit_source", "icu_admit_source", "icu_id", "icu_type", 
                        "apache_3j_bodysystem"]


In [319]:
 print('Transform all String features to category.\n')
    #Convert string features to category data type and perform label encoding
for feature in categorical_features:
    # Convert the feature to string data type
    data_test_clean[feature] = data_test_clean[feature].astype('str')
    data_test[feature] = data_test[feature].astype('str')
    # Fit a LabelEncoder on the unique values of the feature from both the training and test datasets
    le = LabelEncoder().fit(np.unique(data_test_clean[feature].unique().tolist() + data_test[feature].unique().tolist()))
    # Transform the feature values using the fitted LabelEncoder
    data_test_clean[feature] = le.transform(data_test_clean[feature]) + 1
    data_test[feature] = le.transform(data_test[feature]) + 1
    # Replace NaN values with 0 and convert the feature to integer and then category data type
    data_test_clean[feature] = data_test_clean[feature].fillna(0).astype('int').astype('category')
    data_test[feature] = data_test[feature].fillna(0).astype('int').astype('category')


Transform all String features to category.



In [320]:
# Define lists of categorical and numeric columns
categorical_columns = ["hospital_id", "hospital_admit_source", "icu_admit_source", "icu_id", "icu_type", "apache_3j_bodysystem"]
numeric_columns = ['age','elective_surgery','albumin_apache','apache_2_diagnosis','apache_post_operative',
'arf_apache','bilirubin_apache','bun_apache','creatinine_apache','fio2_apache','gcs_unable_apache','glucose_apache',
'heart_rate_apache','hematocrit_apache','intubated_apache','map_apache','paco2_for_ph_apache','pao2_apache','ph_apache',
'resprate_apache','sodium_apache','temp_apache','urineoutput_apache','ventilated_apache','wbc_apache','d1_diasbp_invasive_max',
'd1_diasbp_invasive_min','d1_diasbp_max','d1_diasbp_min','d1_heartrate_max','d1_heartrate_min','d1_mbp_invasive_max',
'd1_mbp_invasive_min','d1_mbp_max','d1_mbp_min','d1_resprate_max','d1_resprate_min','d1_spo2_max','d1_spo2_min',
'd1_sysbp_invasive_max','d1_sysbp_invasive_min','d1_sysbp_max','d1_sysbp_min','d1_temp_max','d1_temp_min',
'h1_diasbp_invasive_max','h1_diasbp_invasive_min','h1_diasbp_max','h1_diasbp_min','h1_heartrate_max','h1_heartrate_min',
'h1_mbp_invasive_max','h1_mbp_invasive_min','h1_mbp_max','h1_mbp_min','h1_mbp_noninvasive_min','h1_resprate_max',
'h1_resprate_min','h1_spo2_max','h1_spo2_min','h1_sysbp_invasive_max','h1_sysbp_invasive_min','h1_sysbp_max','h1_sysbp_min',
'h1_temp_max','h1_temp_min','h1_albumin_max','h1_albumin_min','h1_bilirubin_max','h1_bilirubin_min','h1_bun_max','h1_bun_min',
'h1_calcium_max','h1_calcium_min','h1_creatinine_max','h1_creatinine_min','h1_glucose_max','h1_glucose_min','h1_hco3_max',
'h1_hco3_min','h1_hemaglobin_max','h1_hemaglobin_min','h1_hematocrit_max','h1_hematocrit_min','h1_inr_max','h1_inr_min',
'h1_lactate_max','h1_lactate_min','h1_platelets_max','h1_platelets_min','h1_potassium_max','h1_potassium_min','h1_sodium_max',
'h1_sodium_min','h1_wbc_max','h1_wbc_min','h1_arterial_pco2_max','h1_arterial_pco2_min','h1_arterial_ph_max','h1_arterial_ph_min',
'h1_arterial_po2_max','h1_arterial_po2_min','h1_pao2fio2ratio_max','h1_pao2fio2ratio_min','apache_4a_icu_death_prob','aids',
'cirrhosis','diabetes_mellitus','hepatic_failure','immunosuppression','leukemia','lymphoma','solid_tumor_with_metastasis']


In [321]:
# Remove rows with all missing values from the dataset
data_test_clean = data_test_clean.dropna(how="all")

In [322]:
# Check for missing values in the numeric columns
missing_numeric = data_test_clean[numeric_columns].isna().sum(axis=0)
print("\n\n", '\033[1m'+ 'Look for missing:' + '\033[0m',"\n",missing_numeric.sort_values())



 Look for missing: 
 solid_tumor_with_metastasis        0
elective_surgery                   0
leukemia                           0
apache_post_operative              0
arf_apache                         0
immunosuppression                  0
hepatic_failure                    0
diabetes_mellitus                  0
cirrhosis                          0
ventilated_apache                  0
aids                               0
intubated_apache                   0
lymphoma                           0
d1_mbp_invasive_max               95
d1_mbp_max                        95
d1_mbp_min                        95
d1_mbp_invasive_min               95
d1_diasbp_min                    100
d1_diasbp_max                    100
d1_diasbp_invasive_min           100
d1_diasbp_invasive_max           100
d1_sysbp_invasive_max            100
d1_sysbp_min                     100
d1_sysbp_max                     100
d1_sysbp_invasive_min            100
d1_heartrate_max                 105
d1_heartrate_mi

In [323]:
data_test_clean.head()

,hospital_id,hospital_death,age,elective_surgery,hospital_admit_source,icu_admit_source,icu_id,icu_type,albumin_apache,apache_2_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_unable_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_temp_max,h1_temp_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem
0,60,NaN,56.0,0,4,1,50,7,3.7,301.0,0,0.0,NaN,19.0,0.67,NaN,0.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,74.0,51.0,74.0,51.0,100.0,78.0,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,121.0,79.0,121.0,79.0,37.1,36.6,55.0,51.0,55.0,51.0,88.0,84.0,68.0,NaN,68.0,60.0,60.0,18.0,8.0,100.0,98.0,91.0,79.0,91.0,79.0,36.6,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
1,33,NaN,NaN,1,8,3,28,6,NaN,217.0,1,0.0,NaN,NaN,NaN,NaN,0.0,NaN,60.0,NaN,0.0,53.0,NaN,NaN,NaN,37.0,NaN,37.0,843.4368,0.0,NaN,65.0,31.0,65.0,31.0,73.0,60.0,108.0,53.0,108.0,53.0,33.0,16.0,99.0,90.0,148.0,83.0,148.0,83.0,37.7,36.6,50.0,44.0,50.0,44.0,60.0,60.0,80.0,NaN,80.0,80.0,80.0,16.0,16.0,94.0,92.0,121.0,96.0,121.0,96.0,36.6,36.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
2,64,NaN,66.0,1,8,3,22,1,NaN,302.0,1,0.0,NaN,22.0,1.84,NaN,0.0,81.0,124.0,39.2,0.0,115.0,NaN,NaN,NaN,31.0,130.0,35.5,1728.0000,0.0,14.8,85.0,57.0,85.0,57.0,117.0,86.0,110.0,71.0,110.0,71.0,28.0,10.0,100.0,95.0,172.0,102.0,172.0,102.0,37.5,35.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,26,NaN,64.0,0,8,3,45,6,1.9,303.0,0,0.0,0.8,24.0,0.75,0.7,0.0,122.0,111.0,32.0,1.0,48.0,36.0,142.0,7.35,30.0,133.0,37.7,1434.4128,1.0,9.7,63.0,42.0,63.0,42.0,111.0,60.0,76.0,48.0,76.0,48.0,35.0,15.0,100.0,92.0,115.0,69.0,115.0,69.0,38.4,37.5,62.0,57.0,62.0,57.0,111.0,102.0,76.0,NaN,76.0,65.0,65.0,26.0,23.0,95.0,93.0,115.0,92.0,115.0,92.0,38.0,38.0,1.9,1.9,0.8,0.8,24.0,24.0,7.4,7.4,0.75,0.75,122.0,122.0,22.0,22.0,11.0,11.0,32.0,32.0,1.4,1.4,NaN,NaN,248.0,248.0,4.0,4.0,133.0,133.0,9.7,9.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.17,0.0

In [324]:
labels_to_replace=['age', 'elective_surgery', 'apache_2_diagnosis', 'apache_post_operative', 
                                      'arf_apache', 'gcs_unable_apache', 'heart_rate_apache', 'intubated_apache', 
                                      'map_apache', 'resprate_apache', 'temp_apache', 'ventilated_apache', 
                                      'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min', 'd1_diasbp_max', 'd1_diasbp_min', 
                                      'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min', 
                                      'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 
                                      'd1_spo2_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min', 'd1_sysbp_max', 
                                      'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_invasive_max', 
                                      'h1_diasbp_invasive_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_heartrate_max', 
                                      'h1_heartrate_min', 'h1_mbp_invasive_max', 'h1_mbp_max', 'h1_mbp_min', 
                                      'h1_mbp_noninvasive_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 
                                      'h1_spo2_min', 'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'h1_sysbp_max', 
                                      'h1_sysbp_min', 'apache_4a_icu_death_prob', 'aids', 'cirrhosis', 'diabetes_mellitus', 
                                      'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 
                                      'solid_tumor_with_metastasis']

In [325]:
#Iterate over the labels to replace and perform the replacement
for label in labels_to_replace:
    mask_new = data_test_clean[[label]].notna().all(axis=1)
    data_test_clean = data_test_clean[mask_new]

In [326]:
# Check for missing values in the numeric columns
missing_numeric = data_test_clean[numeric_columns].isna().sum(axis=0)
print("\n\n", '\033[1m'+ 'Look for missing:' + '\033[0m',"\n",missing_numeric.sort_values())



 Look for missing: 
 age                                0
h1_mbp_invasive_max                0
h1_heartrate_min                   0
h1_heartrate_max                   0
h1_diasbp_min                      0
h1_diasbp_max                      0
h1_diasbp_invasive_min             0
h1_mbp_max                         0
h1_diasbp_invasive_max             0
d1_temp_max                        0
d1_sysbp_min                       0
d1_sysbp_max                       0
d1_sysbp_invasive_min              0
d1_sysbp_invasive_max              0
d1_spo2_min                        0
d1_temp_min                        0
d1_spo2_max                        0
h1_mbp_min                         0
lymphoma                           0
leukemia                           0
immunosuppression                  0
hepatic_failure                    0
diabetes_mellitus                  0
cirrhosis                          0
aids                               0
h1_mbp_noninvasive_min             0
apache_4a_icu_d

In [327]:
mask_new = data_test_clean[['glucose_apache']].notna().all(axis=1)
data_test_clean = data_test_clean[mask_new]

In [328]:
mask_new = data_test_clean[['sodium_apache','creatinine_apache','bun_apache']].notna().all(axis=1)
data_test_clean = data_test_clean[mask_new]

In [329]:
mask_new = data_test_clean[['hematocrit_apache','wbc_apache']].notna().all(axis=1)
data_test_clean = data_test_clean[mask_new]

In [330]:
# Check for missing values in the numeric columns
missing_numeric = data_test_clean[numeric_columns].isna().sum(axis=0)
print("\n\n", '\033[1m'+ 'Look for missing:' + '\033[0m',"\n",missing_numeric.sort_values())



 Look for missing: 
 age                                0
h1_mbp_invasive_max                0
h1_heartrate_min                   0
h1_heartrate_max                   0
h1_diasbp_min                      0
h1_diasbp_max                      0
h1_diasbp_invasive_min             0
h1_mbp_max                         0
h1_diasbp_invasive_max             0
d1_temp_max                        0
d1_sysbp_min                       0
d1_sysbp_max                       0
d1_sysbp_invasive_min              0
d1_sysbp_invasive_max              0
d1_spo2_min                        0
d1_temp_min                        0
d1_spo2_max                        0
h1_mbp_min                         0
lymphoma                           0
leukemia                           0
immunosuppression                  0
hepatic_failure                    0
diabetes_mellitus                  0
cirrhosis                          0
aids                               0
h1_mbp_noninvasive_min             0
apache_4a_icu_d

In [331]:
data_test_clean=data_test_clean.drop(['albumin_apache', 'bilirubin_apache', 'fio2_apache', 'paco2_for_ph_apache', 
                                      'pao2_apache', 'ph_apache', 'urineoutput_apache', 'h1_mbp_invasive_min', 
                                      'h1_temp_max', 'h1_temp_min', 'h1_albumin_max', 'h1_albumin_min', 'h1_bilirubin_max', 
                                      'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min', 
                                      'h1_creatinine_max', 'h1_creatinine_min', 'h1_glucose_max', 'h1_glucose_min', 
                                      'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max', 'h1_hemaglobin_min', 
                                      'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max', 
                                      'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min', 'h1_potassium_max', 
                                      'h1_potassium_min', 'h1_sodium_max', 'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min', 
                                      'h1_arterial_pco2_max', 'h1_arterial_pco2_min', 'h1_arterial_ph_max', 
                                      'h1_arterial_ph_min', 'h1_arterial_po2_max', 'h1_arterial_po2_min', 
                                      'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min'], axis=1)

In [332]:
data_test_clean=data_test_clean.drop("hospital_death", axis='columns')

In [333]:
data_test_clean.shape

(19729, 71)

In [334]:
data_test_clean.head()

,hospital_id,age,elective_surgery,hospital_admit_source,icu_admit_source,icu_id,icu_type,apache_2_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_unable_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem
0,60,56.0,0,4,1,50,7,301.0,0,0.0,19.0,0.67,0.0,276.0,105.0,42.0,0.0,57.0,5.0,134.0,36.6,0.0,4.7,74.0,51.0,74.0,51.0,100.0,78.0,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,121.0,79.0,121.0,79.0,37.1,36.6,55.0,51.0,55.0,51.0,88.0,84.0,68.0,68.0,60.0,60.0,18.0,8.0,100.0,98.0,91.0,79.0,91.0,79.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
3,26,64.0,0,8,3,45,6,303.0,0,0.0,24.0,0.75,0.0,122.0,111.0,32.0,1.0,48.0,30.0,133.0,37.7,1.0,9.7,63.0,42.0,63.0,42.0,111.0,60.0,76.0,48.0,76.0,48.0,35.0,15.0,100.0,92.0,115.0,69.0,115.0,69.0,38.4,37.5,62.0,57.0,62.0,57.0,111.0,102.0,76.0,76.0,65.0,65.0,26.0,23.0,95.0,93.0,115.0,92.0,115.0,92.0,0.17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9
4,54,56.0,0,4,1,6,7,217.0,1,0.0,11.0,0.52,0.0,192.0,94.0,32.8,0.0,41.0,5.0,134.0,36.1,1.0,14.5,84.0,24.0,100.0,24.0,94.0,62.0,126.0,37.0,137.0,37.0,21.0,0.0,100.0,95.0,182.0,66.0,222.0,66.0,37.9,36.0,81.0,66.0,81.0,66.0,82.0,76.0,137.0,137.0,113.0,113.0,16.0,16.0,100.0,99.0,222.0,195.0,222.0,194.0,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
5,32,55.0,0,4,1,42,1,113.0,0,0.0,26.0,0.68,0.0,210.0,139.0,46.6,0.0,48.0,36.0,158.0,39.2,0.0,4.5,71.0,43.0,71.0,43.0,139.0,98.0,80.0,51.0,80.0,51.0,34.0,15.0,100.0,93.0,115.0,75.0,115.0,75.0,39.2,37.3,63.0,59.0,63.0,59.0,134.0,133.0,67.0,67.0,67.0,67.0,34.0,28.0,93.0,93.0,103.0,97.0,103.0,97.0,0.09,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10
6,30,79.0,0,15,1,78,6,303.0,0,1.0,40.0,5.56,0.0,106.0,58.0,31.5,0.0,64.0,10.0,136.0,36.9,0.0,4.5,79.0,40.0,79.0,40.0,77.0,58.0,98.0,63.0,98.0,63.0,27.0,9.0,100.0,93.0,194.0,124.0,194.0,124.0,37.0,36.6,58.0,48.0,58.0,48.0,74.0,64.0,86.0,86.0,71.0,71.0,27.0,16.0,100.0,100.0,160.0,136.0,160.0,136.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [335]:
# column_names = data_test_clean.columns.tolist()
# print('Column Names in data_test_clean:')
# for column_name in column_names:
#     print(column_name)

In [336]:
data_test_clean.to_csv(r'C:\Users\brian\OneDrive\Desktop\Documente licență\Documenație scrisă\Date_test_clean.csv',index=False)

In [337]:
electro_data_test_clean=data_test_clean.drop([
'bun_apache','creatinine_apache','hematocrit_apache','sodium_apache',
'wbc_apache',                                           
'apache_4a_icu_death_prob'], axis=1)

In [338]:
electro_data_test_clean.to_csv(r'C:\Users\brian\OneDrive\Desktop\Documente licență\Documenație scrisă\electro_Date_test_clean.csv',index=False)

In [339]:
undf_clean=data_test_clean.isnull().sum()
print("\n\n", '\033[1m'+ 'Look for undefined:' + '\033[0m',"\n",undf_clean.sort_values())



 Look for undefined: 
 hospital_id                    0
h1_mbp_max                     0
h1_mbp_invasive_max            0
h1_heartrate_min               0
h1_heartrate_max               0
h1_diasbp_min                  0
h1_diasbp_max                  0
h1_mbp_min                     0
h1_diasbp_invasive_min         0
d1_temp_min                    0
d1_temp_max                    0
d1_sysbp_min                   0
d1_sysbp_max                   0
d1_sysbp_invasive_min          0
d1_sysbp_invasive_max          0
h1_diasbp_invasive_max         0
d1_spo2_min                    0
h1_mbp_noninvasive_min         0
h1_resprate_min                0
lymphoma                       0
leukemia                       0
immunosuppression              0
hepatic_failure                0
diabetes_mellitus              0
cirrhosis                      0
h1_resprate_max                0
aids                           0
h1_sysbp_min                   0
h1_sysbp_max                   0
h1_sysbp_invasive_